In [83]:
#!pip install mne==1.7.1
#!pip install PyWavelets
#!pip install scipy

In [84]:
import mne
import numpy as np
import pandas as pd
import pywt
from pywt import wavedec
import scipy.stats as stats
import time
import numpy as np
import time
import struct

In [85]:
def prepare_raw_data(raw_electrogram_file):
   raw = mne.io.read_raw_edf(raw_electrogram_file, preload=True) 
   rf = raw.filter(0.1,30)
   
   #selected_channel_name = rf.info['ch_names']
   sample_frequency = rf.info['sfreq']
   epoch_duration = 10
   
   epoch_samples = int(epoch_duration*sample_frequency)
   
   total_epochs = 307200// epoch_samples
   
   #epochs_matrix = np.zeros((total_epochs, 5, epoch_samples))  # 5 channels
   
   # Segmenter les données en epochs de 10 secondes
   events = mne.make_fixed_length_events(rf, duration=epoch_duration)
   # Créer un objet Epochs
   epochs = mne.Epochs(rf, events, tmin=0, tmax=epoch_duration, baseline=None, detrend=1)
   
   # Obtenir les données des epochs sous forme de tableau numpy
   epochs_data = epochs.get_data()
   
   zscores = stats.zscore(epochs_data)
   
   return zscores

In [86]:
def compute_features_cpu(raw_data):
    input_buffer = [i for i in range(len(raw_data[0, 0, :]))]

    # Définition de la famille d'ondelettes à utiliser
    wavelet = 'db4'

    # Initialisation des tableaux pour stocker les caractéristiques
    cD_Energy = np.zeros((raw_data.shape[0], 4))
    cA_Energy = np.zeros((raw_data.shape[0], 4))
    D_Entropy = np.zeros((raw_data.shape[0], 4))
    A_Entropy = np.zeros((raw_data.shape[0], 4))
    D_mean = np.zeros((raw_data.shape[0], 4))
    A_mean = np.zeros((raw_data.shape[0], 4))
    D_std = np.zeros((raw_data.shape[0], 4))
    A_std = np.zeros((raw_data.shape[0], 4))

    for i in range(raw_data.shape[0]):
        for j in range(4):
            input_buffer = raw_data[i, j, :]

            coeffs = pywt.wavedec(input_buffer, wavelet, mode='symmetric', level=5, axis=0)  # Calcul de la DWT avec pywt

            cD_Energy[i,j] = np.mean([np.sum(np.square(coeffs[5])),np.sum(np.square(coeffs[4])),
                                np.sum(np.square(coeffs[3])),np.sum(np.square(coeffs[2])),
                                np.sum(np.square(coeffs[1]))])
            cA_Energy[i,j] = np.sum(np.square(coeffs[0]))
            D_Entropy[i,j] = np.mean([np.sum(np.square(coeffs[5]) * np.log(np.square(coeffs[5]))),
                                np.sum(np.square(coeffs[4]) * np.log(np.square(coeffs[4]))),
                                np.sum(np.square(coeffs[3]) * np.log(np.square(coeffs[3]))),
                                np.sum(np.square(coeffs[2]) * np.log(np.square(coeffs[2]))),
                                np.sum(np.square(coeffs[1]) * np.log(np.square(coeffs[1])))])
            A_Entropy[i,j] = np.sum(np.square(coeffs[0]) * np.log(np.square(coeffs[0])))
            D_mean[i,j] = np.mean([np.mean(coeffs[5]),np.mean(coeffs[4]),np.mean(coeffs[3]),np.mean(coeffs[2]),np.mean(coeffs[1])])
            A_mean[i,j] = np.mean(coeffs[0])
            D_std[i,j] = np.mean([np.std(coeffs[5]),np.std(coeffs[4]),np.std(coeffs[3]),np.std(coeffs[2]),np.std(coeffs[1])])
            A_std[i,j] = np.std(coeffs[0])
            
    return cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std


### FPGA implementation part

In [87]:
"""

from pynq import Overlay
from pynq import allocate
from pynq import PL


class DWC_FEC():
    def __init__(self):
        
        PL.reset()

        #overlay = Overlay('design_3.bit')
        overlay = Overlay('design_float.bit')
        #overlay = Overlay('design_half.bit')
        #overlay = Overlay('design_fixed_32_5.bit')
        #overlay = Overlay('design_fixed_32_16.bit')
        #overlay = Overlay('design_fixed_16_8.bit')
        #overlay = Overlay('design_fixed_24_16.bit')
        #overlay = Overlay('design_fixed_24_12.bit')
        #overlay = Overlay('design_fixed_32_12_vhdl.bit')

        self.dwt_0_hls = True
        self.dwt_1_hls = True

        # Display the names of the overlay IP blocks
        #print('IP blocks :', list(overlay.ip_dict.keys()))

        self.reducer_0     = overlay.reducer_0
        self.reducer_1     = overlay.reducer_1

        if self.dwt_0_hls:
            self.dwt_db4_hls_0 = overlay.dwt_db4_hls_0
        if self.dwt_1_hls:
            self.dwt_db4_hls_1 = overlay.dwt_db4_hls_1

        self.dma_data_in_0 = overlay.axi_dma_data_in_0
        self.dma_data_in_0_send = overlay.axi_dma_data_in_0.sendchannel

        self.dma_data_in_1 = overlay.axi_dma_data_in_1
        self.dma_data_in_1_send = overlay.axi_dma_data_in_1.sendchannel

        self.dma_coeff_lo_0 = overlay.axi_dma_coeff_lo_0
        self.dma_coeff_lo_0_recv = overlay.axi_dma_coeff_lo_0.recvchannel

        self.dma_coeff_lo_1 = overlay.axi_dma_coeff_lo_1
        self.dma_coeff_lo_1_recv = overlay.axi_dma_coeff_lo_1.recvchannel

        #Get register offset of my parameter
        def get_register_offset(overlay, ip, parameter):
            return overlay.ip_dict[ip]['registers'][parameter]['address_offset']

        if self.dwt_0_hls:
            self.DWT_0_SIZE_REGISTER = get_register_offset(overlay, 'dwt_db4_hls_0', 'size')
            #print('DWT_0_SIZE_REGISTER =', hex(self.DWT_0_SIZE_REGISTER))
        if self.dwt_1_hls:
            self.DWT_1_SIZE_REGISTER = get_register_offset(overlay, 'dwt_db4_hls_1', 'size')
            #print('DWT_1_SIZE_REGISTER =', hex(self.DWT_1_SIZE_REGISTER))
        if self.dwt_0_hls:
            self.DEBUG_REGISTER = get_register_offset(overlay, 'dwt_db4_hls_0', 'debug')
            #print('DEBUG_REGISTER =', hex(self.DEBUG_REGISTER))

        self.RED_0_SIZE_REGISTER = get_register_offset(overlay, 'reducer_0', 'size')
        #print('RED_SIZE_REGISTER =', hex(RED_0_SIZE_REGISTER))

        self.RED_1_SIZE_REGISTER = get_register_offset(overlay, 'reducer_1', 'size')
        #print('RED_SIZE_REGISTER =', hex(RED_1_SIZE_REGISTER))

        self.SQUARE_SUM_0_REGISTER = get_register_offset(overlay, 'reducer_0', 'square_sum')
        #print('SQUARE_SUM_0_REGISTER =', hex(SQUARE_SUM_0_REGISTER))

        self.SQUARE_SUM_1_REGISTER = get_register_offset(overlay, 'reducer_1', 'square_sum')
        #print('SQUARE_SUM_1_REGISTER =', hex(SQUARE_SUM_1_REGISTER))

        self.MEAN_0_REGISTER = get_register_offset(overlay, 'reducer_0', 'mean')
        #print('MEAN_0_REGISTER =', hex(MEAN_0_REGISTER))

        self.MEAN_1_REGISTER = get_register_offset(overlay, 'reducer_1', 'mean')
        #print('MEAN_1_REGISTER =', hex(MEAN_1_REGISTER))

        self.STD_0_REGISTER = get_register_offset(overlay, 'reducer_0', 'std')
        #print('STD_0_REGISTER =', hex(STD_0_REGISTER))

        self.STD_1_REGISTER = get_register_offset(overlay, 'reducer_0', 'std')
        #print('STD_1_REGISTER =', hex(STD_1_REGISTER))

        self.ENTROPY_0_REGISTER = get_register_offset(overlay, 'reducer_0', 'entropy')
        #print('ENTROPY_0_REGISTER =', hex(ENTROPY_0_REGISTER))

        self.ENTROPY_1_REGISTER = get_register_offset(overlay, 'reducer_1', 'entropy')
        #print('ENTROPY_1_REGISTER =', hex(ENTROPY_1_REGISTER))

        #Start my IPs

        self.CONTROL_REGISTER = 0x0
        #dwt_db4_hls.write(CONTROL_REGISTER, 0x81)


    def print_dma_data(self, dma_channel):
        print("error: ", dma_channel.error, " idle: ", dma_channel.idle, " running: ", dma_channel.running)

    def process_data(self, in_buffer_0, in_buffer_1, lo_buffer_0, lo_buffer_1):
        size_0 = len(in_buffer_0)
        dsize_0 = int((size_0 + 8 - 1)/2)
        size_1 = len(in_buffer_1)
        dsize_1 = int((size_1 + 8 - 1)/2)
        #print(size)
        #print("debug 1 ", dwt_db4_hls.read(DEBUG_REGISTER))
        
        if self.dwt_0_hls:
            self.dwt_db4_hls_0.write(self.DWT_0_SIZE_REGISTER, size_0)
        if self.dwt_1_hls:
            self.dwt_db4_hls_1.write(self.DWT_1_SIZE_REGISTER, size_1)
        self.reducer_0.write(self.RED_0_SIZE_REGISTER, dsize_0)
        self.reducer_1.write(self.RED_1_SIZE_REGISTER, dsize_1)
        
        if self.dwt_0_hls:
            self.dwt_db4_hls_0.write(self.CONTROL_REGISTER, 0x01)
        if self.dwt_1_hls:
            self.dwt_db4_hls_1.write(self.CONTROL_REGISTER, 0x01)
            
        self.reducer_0.write(self.CONTROL_REGISTER, 0x01)
        self.reducer_1.write(self.CONTROL_REGISTER, 0x01)
        
        #print("debug 2 ", dwt_db4_hls.read(DEBUG_REGISTER))
        self.dma_coeff_lo_0_recv.transfer(lo_buffer_0)
        self.dma_coeff_lo_1_recv.transfer(lo_buffer_1)  
        self.dma_data_in_0_send.transfer(in_buffer_0)  
        self.dma_data_in_1_send.transfer(in_buffer_1)   
        self.dma_data_in_0_send.wait()
        self.dma_data_in_1_send.wait()
        self.dma_coeff_lo_0_recv.wait()
        self.dma_coeff_lo_1_recv.wait()
        #print("debug 3 ", dwt_db4_hls.read(DEBUG_REGISTER))
        #dwt_db4_hls.write(CONTROL_REGISTER, 0x00)
        square_sum_0 = struct.unpack('f', struct.pack('I', self.reducer_0.read(self.SQUARE_SUM_0_REGISTER)))[0]
        mean_0 = struct.unpack('f', struct.pack('I', self.reducer_0.read(self.MEAN_0_REGISTER)))[0]
        std_0 = struct.unpack('f', struct.pack('I', self.reducer_0.read(self.STD_0_REGISTER)))[0]
        entropy_0 = struct.unpack('f', struct.pack('I', self.reducer_0.read(self.ENTROPY_0_REGISTER)))[0]
        square_sum_1 = struct.unpack('f', struct.pack('I', self.reducer_1.read(self.SQUARE_SUM_1_REGISTER)))[0]
        mean_1 = struct.unpack('f', struct.pack('I', self.reducer_1.read(self.MEAN_1_REGISTER)))[0]
        std_1 = struct.unpack('f', struct.pack('I', self.reducer_1.read(self.STD_1_REGISTER)))[0]
        entropy_1 = struct.unpack('f', struct.pack('I', self.reducer_1.read(self.ENTROPY_1_REGISTER)))[0]
        return [square_sum_0, mean_0, std_0, entropy_0, square_sum_1, mean_1, std_1, entropy_1]
    
    def compute_features_fpga(self, raw_data):
        
        cD_Energy_pynq = np.zeros((59, 5))
        cA_Energy_pynq = np.zeros((59, 5))
        D_Entropy_pynq = np.zeros((59, 5))
        A_Entropy_pynq = np.zeros((59, 5))
        D_mean_pynq = np.zeros((59, 5))
        A_mean_pynq = np.zeros((59, 5))
        D_std_pynq = np.zeros((59, 5))
        A_std_pynq = np.zeros((59, 5))
        
        #create buffers
        data_in_size = len(raw_data[0, 0, :])
        data_1_size = int((data_in_size + 8 - 1) / 2)
        data_2_size = int((data_1_size + 8 - 1) / 2)
        data_3_size = int((data_2_size + 8 - 1) / 2)
        data_4_size = int((data_3_size + 8 - 1) / 2)
        data_5_size = int((data_4_size + 8 - 1) / 2)
        print(data_in_size)
        print(data_1_size)
        print(data_2_size)
        print(data_3_size)
        print(data_4_size)
        print(data_5_size)

        input_buffer_0 = allocate(shape=(data_in_size,), dtype=np.float32)
        lo_1_buffer_0 = allocate(shape=(data_1_size,), dtype=np.float32)
        lo_2_buffer_0 = allocate(shape=(data_2_size,), dtype=np.float32)
        lo_3_buffer_0 = allocate(shape=(data_3_size,), dtype=np.float32)
        lo_4_buffer_0 = allocate(shape=(data_4_size,), dtype=np.float32)
        lo_5_buffer_0 = allocate(shape=(data_5_size,), dtype=np.float32)

        input_buffer_1 = allocate(shape=(data_in_size,), dtype=np.float32)
        lo_1_buffer_1 = allocate(shape=(data_1_size,), dtype=np.float32)
        lo_2_buffer_1 = allocate(shape=(data_2_size,), dtype=np.float32)
        lo_3_buffer_1 = allocate(shape=(data_3_size,), dtype=np.float32)
        lo_4_buffer_1 = allocate(shape=(data_4_size,), dtype=np.float32)
        lo_5_buffer_1 = allocate(shape=(data_5_size,), dtype=np.float32)

        for i in range(raw_data.shape[0]):
            for j in range(2):

                input_buffer_0[0:len(raw_data[i, j, :])] = raw_data[i, j, :]
                input_buffer_1[0:len(raw_data[i, j + 2, :])] = raw_data[i, j + 2, :]

                squared_sum_1_0, mean_1_0, std_1_0, entropy_1_0, squared_sum_1_1, mean_1_1, std_1_1, entropy_1_1 = self.process_data(input_buffer_0, input_buffer_1, lo_1_buffer_0, lo_1_buffer_1)
                squared_sum_2_0, mean_2_0, std_2_0, entropy_2_0, squared_sum_2_1, mean_2_1, std_2_1, entropy_2_1 = self.process_data(lo_1_buffer_0,  lo_1_buffer_1,  lo_2_buffer_0, lo_2_buffer_1)
                squared_sum_3_0, mean_3_0, std_3_0, entropy_3_0, squared_sum_3_1, mean_3_1, std_3_1, entropy_3_1 = self.process_data(lo_2_buffer_0,  lo_2_buffer_1,  lo_3_buffer_0, lo_3_buffer_1)
                squared_sum_4_0, mean_4_0, std_4_0, entropy_4_0, squared_sum_4_1, mean_4_1, std_4_1, entropy_4_1 = self.process_data(lo_3_buffer_0,  lo_3_buffer_1,  lo_4_buffer_0, lo_4_buffer_1)
                squared_sum_5_0, mean_5_0, std_5_0, entropy_5_0, squared_sum_5_1, mean_5_1, std_5_1, entropy_5_1 = self.process_data(lo_4_buffer_0,  lo_4_buffer_1,  lo_5_buffer_0, lo_5_buffer_1)

                cD_Energy_pynq[i,j] = (squared_sum_1_0 + squared_sum_2_0 + squared_sum_3_0 + squared_sum_4_0 + squared_sum_5_0)/5
                cA_Energy_pynq[i,j] = np.sum(np.square(lo_5_buffer_0))
                D_Entropy_pynq[i,j] = (entropy_1_0 + entropy_2_0 + entropy_3_0 + entropy_4_0 + entropy_5_0)/5
                A_Entropy_pynq[i,j] = np.sum(np.square(lo_5_buffer_0) * np.log(np.square(lo_5_buffer_0)))
                D_mean_pynq[i,j]    = (mean_1_0 + mean_2_0 + mean_3_0 + mean_4_0 + mean_5_0)/5
                A_mean_pynq[i,j]    = np.mean(lo_5_buffer_0)
                D_std_pynq[i,j]    = (std_1_0 + std_2_0 + std_3_0 + std_4_0 + std_5_0)/5
                A_std_pynq[i,j]    = np.std(lo_5_buffer_0)
                
                cD_Energy_pynq[i,j+2] = (squared_sum_1_1 + squared_sum_2_1 + squared_sum_3_1 + squared_sum_4_1 + squared_sum_5_1)/5
                cA_Energy_pynq[i,j+2] = np.sum(np.square(lo_5_buffer_1))
                D_Entropy_pynq[i,j+2] = (entropy_1_1 + entropy_2_1 + entropy_3_1 + entropy_4_1 + entropy_5_1)/5
                A_Entropy_pynq[i,j+2] = np.sum(np.square(lo_5_buffer_1) * np.log(np.square(lo_5_buffer_1)))
                D_mean_pynq[i,j+2]    = (mean_1_1 + mean_2_1 + mean_3_1 + mean_4_1 + mean_5_1)/5
                A_mean_pynq[i,j+2]    = np.mean(lo_5_buffer_1)
                D_std_pynq[i,j+2]    = (std_1_1 + std_2_1 + std_3_1 + std_4_1 + std_5_1)/5
                A_std_pynq[i,j+2]    = np.std(lo_5_buffer_1)
                
        return cD_Energy_pynq, cA_Energy_pynq, D_Entropy_pynq, A_Entropy_pynq, D_mean_pynq, A_mean_pynq, D_std_pynq, A_std_pynq
        
"""        

'\n\nfrom pynq import Overlay\nfrom pynq import allocate\nfrom pynq import PL\n\n\nclass DWC_FEC():\n    def __init__(self):\n        \n        PL.reset()\n\n        #overlay = Overlay(\'design_3.bit\')\n        overlay = Overlay(\'design_float.bit\')\n        #overlay = Overlay(\'design_half.bit\')\n        #overlay = Overlay(\'design_fixed_32_5.bit\')\n        #overlay = Overlay(\'design_fixed_32_16.bit\')\n        #overlay = Overlay(\'design_fixed_16_8.bit\')\n        #overlay = Overlay(\'design_fixed_24_16.bit\')\n        #overlay = Overlay(\'design_fixed_24_12.bit\')\n        #overlay = Overlay(\'design_fixed_32_12_vhdl.bit\')\n\n        self.dwt_0_hls = True\n        self.dwt_1_hls = True\n\n        # Display the names of the overlay IP blocks\n        #print(\'IP blocks :\', list(overlay.ip_dict.keys()))\n\n        self.reducer_0     = overlay.reducer_0\n        self.reducer_1     = overlay.reducer_1\n\n        if self.dwt_0_hls:\n            self.dwt_db4_hls_0 = overlay.dwt_db4

In [88]:
"""
def compute_rmse(cpu_feature, fpga_feature):
    rmse = 0.0
    for i in range(cpu_feature.shape[0]):
        for j in range(4):
            if(fpga_feature[i,j] != fpga_feature[i,j]):
                print("some nand in fpga_feature")
                continue
    
            rmse += (cpu_feature[i,j] - fpga_feature[i,j])**2
    rmse = np.sqrt(rmse/(cpu_feature.shape[0]*4))
    
    return rmse

def compute_mean_percentual_error(cpu_feature, fpga_feature):
    mean_error = 0.0
    for i in range(cpu_feature.shape[0]):
        for j in range(4):
            if(fpga_feature[i,j] != fpga_feature[i,j]):
                print("some nand in fpga_feature")
                continue
    
            mean_error += np.abs((cpu_feature[i,j] - fpga_feature[i,j])/cpu_feature[i,j])
    mean_error = mean_error/(cpu_feature.shape[0]*4)
    
    return mean_error
    
cD_Energy_rmse = compute_rmse(cD_Energy, cD_Energy_pynq)
cA_Energy_rmse = compute_rmse(cA_Energy, cA_Energy_pynq)
D_Entropy_rmse = compute_rmse(D_Entropy, D_Entropy_pynq)
A_Entropy_rmse = compute_rmse(A_Entropy, A_Entropy_pynq)
D_mean_rmse = compute_rmse(D_mean, D_mean_pynq)
A_mean_rmse = compute_rmse(A_mean, A_mean_pynq)
D_std_rmse = compute_rmse(D_std, D_std_pynq)
A_std_rmse = compute_rmse(A_std, A_std_pynq)

cD_Energy_mpe = compute_mean_percentual_error(cD_Energy, cD_Energy_pynq)
cA_Energy_mpe = compute_mean_percentual_error(cA_Energy, cA_Energy_pynq)
D_Entropy_mpe = compute_mean_percentual_error(D_Entropy, D_Entropy_pynq)
A_Entropy_mpe = compute_mean_percentual_error(A_Entropy, A_Entropy_pynq)
D_mean_mpe = compute_mean_percentual_error(D_mean, D_mean_pynq)
A_mean_mpe = compute_mean_percentual_error(A_mean, A_mean_pynq)
D_std_mpe = compute_mean_percentual_error(D_std, D_std_pynq)
A_std_mpe = compute_mean_percentual_error(A_std, A_std_pynq)

print("cD_Energy rmse: ", cD_Energy_rmse)
print("cA_Energy rmse: ", cA_Energy_rmse)
print("D_Entropy rmse: ", D_Entropy_rmse)
print("A_Entropy rmse: ", A_Entropy_rmse)
print("D_mean rmse: ", D_mean_rmse)
print("A_mean rmse: ", A_mean_rmse)
print("D_std rmse: ", D_std_rmse)
print("A_std rmse: ", A_std_rmse)

print("cD_Energy mean % error: ", cD_Energy_mpe)
print("cA_Energy rmse: ", cA_Energy_mpe)
print("D_Entropy mean % error: ", D_Entropy_mpe)
print("A_Entropy rmse: ", A_Entropy_mpe)
print("D_mean mean % error: ", D_mean_mpe)
print("A_mean rmse: ", A_mean_mpe)
print("D_std mean % error: ", D_std_mpe)
print("A_std rmse: ", A_std_mpe)
"""

'\ndef compute_rmse(cpu_feature, fpga_feature):\n    rmse = 0.0\n    for i in range(cpu_feature.shape[0]):\n        for j in range(4):\n            if(fpga_feature[i,j] != fpga_feature[i,j]):\n                print("some nand in fpga_feature")\n                continue\n    \n            rmse += (cpu_feature[i,j] - fpga_feature[i,j])**2\n    rmse = np.sqrt(rmse/(cpu_feature.shape[0]*4))\n    \n    return rmse\n\ndef compute_mean_percentual_error(cpu_feature, fpga_feature):\n    mean_error = 0.0\n    for i in range(cpu_feature.shape[0]):\n        for j in range(4):\n            if(fpga_feature[i,j] != fpga_feature[i,j]):\n                print("some nand in fpga_feature")\n                continue\n    \n            mean_error += np.abs((cpu_feature[i,j] - fpga_feature[i,j])/cpu_feature[i,j])\n    mean_error = mean_error/(cpu_feature.shape[0]*4)\n    \n    return mean_error\n    \ncD_Energy_rmse = compute_rmse(cD_Energy, cD_Energy_pynq)\ncA_Energy_rmse = compute_rmse(cA_Energy, cA_Energ

In [89]:
def prepare_dataframe(cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std, state):
    df = pd.DataFrame(cD_Energy)
    df1 = pd.DataFrame(cA_Energy)
    df2 = pd.DataFrame(D_Entropy)
    df3 = pd.DataFrame(A_Entropy)
    df4 = pd.DataFrame(D_mean)
    df5 = pd.DataFrame(A_mean)
    df6 = pd.DataFrame(D_std)
    df7 = pd.DataFrame(A_std)
    
    ddd = pd.concat([df,df1,df2,df3,df4,df5,df6,df7], axis = 1)
    
    dddd = np.array(ddd)
    #head = np.array(['stage','cD_energy_fz','cD_energy_cz','cD_energy_c3','cD_energy_c4','cD_energy_pz','cA_energy_fz','cA_energy_cz','cA_energy_c3','cA_energy_c4','cA_energy_pz','D_entropy_fz','D_entropy_cz','D_entropy_c3','D_entropy_c4','D_entropy_pz','A_entropy_fz','A_entropy_cz','A_entropy_c3','A_entropy_c4','A_entropy_pz','D_mean_fz','D_mean_cz','D_mean_c3','D_mean_c4','D_mean_pz','A_mean_fz','A_mean_cz','A_mean_c3','A_mean_c4','A_mean_pz','D_std_fz','D_std_cz','D_std_c3','D_std_c4','D_std_pz','A_std_fz','A_std_cz','A_std_c3','A_std_c4','A_std_pz'])
    rr = pd.DataFrame(dddd)
    
    idx = 0
    #new_col = ['2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2','2']
    #rr.insert(loc=idx, column='stage', value=new_col)
    rr.insert(loc=idx, column='stage', value=state)
    
    return rr

In [90]:
#%%time
raw_data = prepare_raw_data("2-1.edf")
#print(raw_data.shape)
cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std = compute_features_cpu(raw_data)
dataframe_1 = prepare_dataframe(cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std, 1)

raw_data = prepare_raw_data("2-2.edf")
#print(raw_data.shape)
cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std = compute_features_cpu(raw_data)
dataframe_2 = prepare_dataframe(cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std, 2)

raw_data = prepare_raw_data("2-3.edf")
#print(raw_data.shape)
cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std = compute_features_cpu(raw_data)
dataframe_3 = prepare_dataframe(cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std, 2)

df = pd.concat([dataframe_1, dataframe_2, dataframe_3], ignore_index=True)

Extracting EDF parameters from /home/emanuel/workspace/DWT_FPGA/notebooks/2-1.edf...
EDF file detected


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 307199  =      0.000 ...   599.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 16897 samples (33.002 s)

Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 60 events and 5121 original time points ...
1 bad epochs dropped
Extracting EDF parameters from /home/emanuel/workspace/DWT_FPGA/notebooks/2-2.edf...
EDF file detected
Setti

In [91]:
df['stage'].value_counts()

stage
2    118
1     59
Name: count, dtype: int64

In [92]:
df['stage'].value_counts()/float(len(df))

stage
2    0.666667
1    0.333333
Name: count, dtype: float64

In [93]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score,precision_score,recall_score,classification_report, confusion_matrix,cohen_kappa_score,fbeta_score,roc_curve,auc,average_precision_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.utils import resample


In [94]:
# Separate majority and minority classes
df_majority = df[df.stage==2]
df_minority = df[df.stage==1]

# Upsample minority class
df_minority_upsampled = resample(df_minority,
                                replace=True,     # sample with replacement
                                n_samples=120,    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
#df = pd.concat([df_majority, df_minority_upsampled])

In [95]:
df['stage'].value_counts()

stage
2    118
1     59
Name: count, dtype: int64

In [96]:
df['stage'].value_counts()/float(len(df))

stage
2    0.666667
1    0.333333
Name: count, dtype: float64

In [97]:
X = df.drop(['stage'], axis=1)
y = df['stage']

X_train_norm, X_test_norm, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 10, shuffle= True)

scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train_norm)

X_test_norm = scaler.transform(X_test_norm)

rbf_svm = SVC(C=1,kernel='rbf') # rbf_svm is a RBF support vectors

rbf_svm.fit(X_train_norm,y_train)

y_rbf_svm=rbf_svm.predict(X_test_norm)

print ('************* Peformance Evauation of RBF SVM **************')
print(confusion_matrix(y_test,y_rbf_svm))
print(classification_report(y_test,y_rbf_svm))

************* Peformance Evauation of RBF SVM **************
[[15  0]
 [ 0 39]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00        39

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

